# Colab環境でRailsのコーディングテストを行う

- Qiita記事: https://qiita.com/jerrywdlee/items/a3000990c50933e66871
- ソースコード: https://github.com/jerrywdlee/coding-test-rails

## 環境構築

In [0]:
# Ruby環境構築
!apt update -y
!apt install -y -qq ruby

# 以降はRails用
!apt install -y -qq ruby-dev make sqlite3 libsqlite3-dev
# !apt install libtool libffi-dev libzmq3-dev libczmq-dev
!gem install --no-ri --no-rdoc bundler

In [2]:
# SQL文のテストもスムーズに行えるためのSQlite環境設定
%%bash

cat << SQLITERC > ~/.sqliterc
.header on
.mode column
SQLITERC

cat ~/.sqliterc

.header on
.mode column


In [0]:
# RailsサンプルのClone
!git clone https://github.com/jerrywdlee/coding-test-rails.git

In [4]:
# デフォルトのDirの移動(「!」がいらない)
cd coding-test-rails

/content/coding-test-rails


In [0]:
# Rails環境構築(遅い)
!bundle install
# !bundle exec rails db:drop
!bundle exec rails db:create
!bundle exec rake db:migrate

In [6]:
# Rails環境確認
%%bash

cat << RUBY > tmp/script.rb
puts "Rails v#{Rails.version}"
puts "Env: #{Rails.env}"
RUBY

# cat tmp/script.rb
bundle exec rails runner tmp/script.rb

Rails v6.0.0
Env: development


Running via Spring preloader in process 15336


In [7]:
# SQL環境確認
%%bash

cat << SQL > tmp/sql_code.sql
SELECT 1+1;
SQL

# cat tmp/sql_code.sql
cat tmp/sql_code.sql | bundle exec rails db

1+1       
----------
2         


In [8]:
# データCSVの作成(Python)
import seaborn as sns

# タイタニック号の生死者データ
titanic = sns.load_dataset("titanic")
display (titanic.head(5))

titanic.to_csv("tmp/titanic.csv", index=False)
# titanic.head(100).to_csv("tmp/titanic.csv", index=False)

!ls -la tmp/*.csv
!wc -l tmp/*.csv

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


-rw-r--r-- 1 root root 57018 Sep 13 06:04 tmp/titanic.csv
892 tmp/titanic.csv


In [9]:
# サンプルCSVをRailsにインポート
!bundle exec rails runner 'DataImport.load_csv "tmp/titanic.csv"'

Running via Spring preloader in process 15368
891 passengers inseted!


## 質問

### 1.タイタニック号乗客の生還率を求めなさい
*生還率: 生存した乗客数 / 乗客の総数*  

*ソースコードは下記のリンクで確認できます*  
*https://github.com/jerrywdlee/coding-test-rails*

In [10]:
# Rails Ver.
%%bash

cat << RUBY > tmp/script.rb

# ここからはRailsコードです

alive_cnt = Passenger.alive.size
ttl_cnt =  Passenger.all.size
puts "Survival Rate: #{(alive_cnt.to_f * 100 / ttl_cnt).round(1)}%"

# ここまではRailsコードです

RUBY

bundle exec rails runner tmp/script.rb

Survival Rate: 38.4%


Running via Spring preloader in process 15418


In [11]:
# SQL Ver.
%%bash

cat << SQL > tmp/sql_code.sql

-- ここからはSQL文です

SELECT
  (alive_cnt * 1.0 / ttl_cnt) AS survival_rate
FROM
(
  SELECT COUNT(*) AS alive_cnt FROM passengers WHERE survived = 1
) AS alive,
(
  SELECT COUNT(*) AS ttl_cnt FROM passengers
) AS ttl;

-- ここまではSQL文です

SQL

cat tmp/sql_code.sql | bundle exec rails db

survival_rate    
-----------------
0.383838383838384
